In [1]:
import pandas as pd
import os
import datetime
import pickle
import re
import time
from collections import Counter
import numpy as np

import nltk
nltk.data.path

['/home/ec2-user/nltk_data',
 '/home/ec2-user/SageMaker/.conda/envs/my_py/nltk_data',
 '/home/ec2-user/SageMaker/.conda/envs/my_py/share/nltk_data',
 '/home/ec2-user/SageMaker/.conda/envs/my_py/lib/nltk_data',
 '/usr/share/nltk_data',
 '/usr/local/share/nltk_data',
 '/usr/lib/nltk_data',
 '/usr/local/lib/nltk_data']

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [4]:
from ast import literal_eval
from collections import Counter

## 1. Import Regulatory Sections and Noun Chunks with Areas

In [5]:
# Noun chunks with areas
nounchunks_area=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/DictionaryOfRegulatoryNounChunks.csv')
print(nounchunks_area.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10645 entries, 0 to 10644
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nc_code      10645 non-null  int64 
 1   noun_chunks  10645 non-null  object
 2   rin          10645 non-null  object
 3   area         10185 non-null  object
 4   area_no      10645 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 415.9+ KB
None


In [6]:
nounchunks_area.head()

,nc_code,noun_chunks,rin,area,area_no
0,0,180-day exclusivity,0910-AC11,{1},1
1,1,1983 amendment,2115-AB72,{2},1
2,2,1988 amendment,1205-AB05,{4},1
3,3,1990 farm bill,0584-AB28,{1},1
4,4,1993 provision,"0970-AB32,3206-AG31",{1},1


In [7]:
# Convert to dictionary
nounchunks_area=nounchunks_area[nounchunks_area['area_no']>0].set_index('noun_chunks')
nounchunks_area_dict=nounchunks_area.to_dict()['area']
print(len(nounchunks_area_dict))

10185


In [8]:
# Expanded reg sentences with matched noun chunks
df_regSentsExpand=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/RegSentsExpand_NounChunks3.pkl')
print(df_regSentsExpand.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822737 entries, 0 to 822736
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           822737 non-null  object
 1   RegSentsExpand               822737 non-null  object
 2   NounChunksMatch              822737 non-null  int64 
 3   NounChunkMatchWords          822737 non-null  object
 4   NounChunkMatchWordsFiltered  822737 non-null  object
 5   NounChunkMatchFiltered       822737 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 37.7+ MB
None


In [9]:
# Remove duplicated articles
IDs_nodup=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/IDs_no_duplicates.csv')
print(IDs_nodup.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788516 entries, 0 to 788515
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   ID      788516 non-null  int64
dtypes: int64(1)
memory usage: 6.0 MB
None


In [10]:
df_regSentsExpand['ID']=df_regSentsExpand['ID'].astype('int64')
df_regSentsExpand=IDs_nodup.merge(df_regSentsExpand,on='ID',how='left').reset_index(drop=True)
print(df_regSentsExpand.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788516 entries, 0 to 788515
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           788516 non-null  int64 
 1   RegSentsExpand               788516 non-null  object
 2   NounChunksMatch              788516 non-null  int64 
 3   NounChunkMatchWords          788516 non-null  object
 4   NounChunkMatchWordsFiltered  788516 non-null  object
 5   NounChunkMatchFiltered       788516 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 36.1+ MB
None


In [11]:
# Refine to reg relevant articles
df_regSentsExpandRelevant=df_regSentsExpand[df_regSentsExpand['NounChunkMatchFiltered']>0].reset_index(drop=True)
print(df_regSentsExpandRelevant.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493418 entries, 0 to 493417
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           493418 non-null  int64 
 1   RegSentsExpand               493418 non-null  object
 2   NounChunksMatch              493418 non-null  int64 
 3   NounChunkMatchWords          493418 non-null  object
 4   NounChunkMatchWordsFiltered  493418 non-null  object
 5   NounChunkMatchFiltered       493418 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 22.6+ MB
None


## 2. Link Expanded Reg Sentences to Areas

### Approch 1 - dominant noun chunk area (dnca): use all unique areas from dominant noun chunks

In [ ]:
df_regSentsExpandRelevant['DominantNounChunk']=''
df_regSentsExpandRelevant['DominantNounChunkArea']=''
for i in range(0, len(df_regSentsExpandRelevant)):
    nc_list=df_regSentsExpandRelevant['NounChunkMatchWordsFiltered'][i]
    dominant_nc=[j for j in Counter(nc_list).keys() if nc_list.count(j)==max(Counter(nc_list).values())]
    dominant_nc_area=set()
    for nc in dominant_nc:
        if nc in nounchunks_area_dict:
            area=literal_eval(nounchunks_area_dict[nc])
            dominant_nc_area.update(area)
    df_regSentsExpandRelevant['DominantNounChunk'][i]=dominant_nc
    df_regSentsExpandRelevant['DominantNounChunkArea'][i]=dominant_nc_area

/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
print(df_regSentsExpandRelevant.info())

### Approach 2 - dominant area (da): use the dominant areas from all noun chunks

In [ ]:
df_regSentsExpandRelevant['AllAreas']=''
for i in range(0, len(df_regSentsExpandRelevant)):
    nounchunks=df_regSentsExpandRelevant['NounChunkMatchWordsFiltered'][i]
    area_list=[]
    for nc in nounchunks:
        if nc in nounchunks_area_dict:
            area=sorted(literal_eval(nounchunks_area_dict[nc]))
            area_list=area_list+area
    df_regSentsExpandRelevant['AllAreas'][i]=area_list

print(df_regSentsExpandRelevant.info())

In [ ]:
df_regSentsExpandRelevant['AreaCount']=''
df_regSentsExpandRelevant['DominantArea']=''
for i in range(0, len(df_regSentsExpandRelevant)):
    area_list=df_regSentsExpandRelevant['AllAreas'][i]
    area_count=Counter(area_list).most_common()
    dominant_area=[j for j in Counter(area_list).keys() if area_list.count(j)==max(Counter(area_list).values())]
    df_regSentsExpandRelevant['AreaCount'][i]=area_count
    df_regSentsExpandRelevant['DominantArea'][i]=dominant_area

In [ ]:
print(df_regSentsExpandRelevant.info())

### Approach 3 - unique distinct area (uda): use all unique areas from area-specific noun chunks

In [ ]:
df_regSentsExpandRelevant['AllDistinctAreas']=''
df_regSentsExpandRelevant['UniqueDistinctAreas']=''
for i in range(0, len(df_regSentsExpandRelevant)):
    nounchunks=df_regSentsExpandRelevant['NounChunkMatchWordsFiltered'][i]
    area_list=[]
    area_set=set()
    for nc in nounchunks:
        if nc in nounchunks_area_dict:
            area=sorted(literal_eval(nounchunks_area_dict[nc]))
            if len(area)==1:
                area_list=area_list+area
    area_set=set(area_list)
    df_regSentsExpandRelevant['AllDistinctAreas'][i]=area_list
    df_regSentsExpandRelevant['UniqueDistinctAreas'][i]=area_set

In [ ]:
print(df_regSentsExpandRelevant.info())

### Approach 4: dominant distinct area (dda): use the dominant areas from area-specific noun chunks

In [ ]:
df_regSentsExpandRelevant['DistinctAreaCount']=''
df_regSentsExpandRelevant['DominantDistinctArea']=''
for i in range(0, len(df_regSentsExpandRelevant)):
    area_list=df_regSentsExpandRelevant['AllDistinctAreas'][i]
    area_count=Counter(area_list).most_common()
    dominant_area=[j for j in Counter(area_list).keys() if area_list.count(j)==max(Counter(area_list).values())]
    df_regSentsExpandRelevant['DistinctAreaCount'][i]=area_count
    df_regSentsExpandRelevant['DominantDistinctArea'][i]=dominant_area
print(df_regSentsExpandRelevant.info())

In [ ]:
print(df_regSentsExpandRelevant.head())

In [ ]:
df_regSentsExpandRelevant.to_pickle('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegSentsExpand_NounChunks_Area.pkl')

## 3. Filtered Noun Chunk Occurences by Area

In [ ]:
# Reg relevant articles
df_regSentsExpandRelevant=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegSentsExpand_NounChunks_Area.pkl')
print(df_regSentsExpandRelevant.info())

In [ ]:
# Filtered noun chunk occurences across regulation-related articles
df_nounchunk_occurences=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/RegSentsExpand_FilteredNounChunkOccurences.csv')
print(df_nounchunk_occurences.info())

In [ ]:
# Dummies by dominant distinct area (dda)
for i in range(1,15):
    var='DominantDistinctArea'+str(i)
    df_regSentsExpandRelevant[var]=0
    for j in range(0, len(df_regSentsExpandRelevant)):
        if i in df_regSentsExpandRelevant['DominantDistinctArea'][j]:
            df_regSentsExpandRelevant[var][j]=1

In [ ]:
# Filtered noun chunks across regulation-related articles by area
for i in range(1,15):
    allMatchWords=[]
    for list in df_regSentsExpandRelevant[df_regSentsExpandRelevant['DominantDistinctArea'+str(i)]==1]['NounChunkMatchWordsFiltered']:
        allMatchWords=allMatchWords+list
    allMatchWordsCount=Counter(allMatchWords)
    var_name='Occurences_dda'+str(i)
    df_MatchWords = pd.DataFrame(allMatchWordsCount.items(),columns = ['Noun Chunks',var_name])
    df_nounchunk_occurences=df_nounchunk_occurences.merge(df_MatchWords,on='Noun Chunks',how='outer')
print(df_nounchunk_occurences)

In [ ]:
#df_nounchunk_occurences.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_FilteredNounChunkOccurences.csv',index=False)

In [ ]:
# df_nounchunk_occurences=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_FilteredNounChunkOccurences.csv')
# print(df_nounchunk_occurences.info())

In [ ]:
# # Combine Areas 8 & 9
# df_regSentsExpandRelevant['DominantDistinctArea8_9']=0
# df_regSentsExpandRelevant.loc[(df_regSentsExpandRelevant['DominantDistinctArea8']==1) | (df_regSentsExpandRelevant['DominantDistinctArea9']==1),
#                              'DominantDistinctArea8_9']=1
# print(df_regSentsExpandRelevant[['DominantDistinctArea8_9']])

In [ ]:
# # Filtered noun chunks across regulation-related articles for area8_9
# allMatchWords=[]
# for list in df_regSentsExpandRelevant[df_regSentsExpandRelevant['DominantDistinctArea8_9']==1]['NounChunkMatchWordsFiltered']:
#     allMatchWords=allMatchWords+list
# allMatchWordsCount=Counter(allMatchWords)
# var_name='Occurences_dda8_9'
# df_MatchWords = pd.DataFrame(allMatchWordsCount.items(),columns = ['Noun Chunks',var_name])
# df_nounchunk_occurences=df_nounchunk_occurences.merge(df_MatchWords,on='Noun Chunks',how='outer')
# print(df_nounchunk_occurences)

In [ ]:
df_nounchunk_occurences.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_FilteredNounChunkOccurences.csv',index=False)

## A1. Examine the Results

In [ ]:
# Investigate the results
df=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegSentsExpand_NounChunks_Area.pkl')

In [ ]:
print(df.info())

In [ ]:
for i in range(50000,55000):
    if len(df['AllDistinctAreas'][i])>1 and len(df['DominantDistinctArea'][i])==1:
        print(df['ID'][i], df['RegSentsExpand'][i],df['NounChunkMatchWordsFiltered'][i],df['AllDistinctAreas'][i], df['DominantDistinctArea'][i])
        print('\n')

In [ ]:
print(df[df['ID']=='294286922']['NounChunkMatchWordsFiltered'].values)
print(df[df['ID']=='294286922'][['AllAreas','DominantArea']])

In [ ]:
for i in range(200000,205000):
    if len(df['AllDistinctAreas'][i])>1 and len(df['DominantDistinctArea'][i])==1 and (len(df['RegSentsExpand'][i])<600):
        print(df['ID'][i], df['RegSentsExpand'][i],df['NounChunkMatchWordsFiltered'][i],df['AllDistinctAreas'][i], df['DominantDistinctArea'][i],'\n')

In [ ]:
print(df[df['ID']=='282489466']['NounChunkMatchWordsFiltered'].values)